In [1]:
import pickle

import numpy as np
import pandas as pd 

from importlib import reload

import helper_funcs;reload(helper_funcs);from helper_funcs import *
import make_models2;reload(make_models2);from make_models2 import *

from functools import partial
from sklearn.decomposition import PCA

from matplotlib import pyplot as plt

from datetime import datetime

import shutil
from functools import partial
from tqdm import tqdm
tqdm = partial(tqdm, position=0, leave=True)

/mmfs1/data/liacz/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
BRAIN_data = np.load(os.path.expanduser('~/Documents/Bilingualism_CVAE/data/array_brains/arr_combine/BRAIN_data.npy'))

In [3]:
latent_dim=32;batch_size=32;disentangle=False;gamma=100
# encoder, decoder, vae = get_MRI_VAE_3D(input_shape=(64, 64, 64, 1), latent_dim=latent_dim, batch_size=batch_size, disentangle=disentangle, gamma=gamma)

encoder, decoder, vae = get_MRI_VAE_3D(input_shape=(64,64,64,1), 
                                            latent_dim=32, 
                                            batch_size = batch_size, 
                                            disentangle=True,
                                            gamma=100,
                                            kernel_size = 3,
                                            filters = 48,
                                            intermediate_dim = 128,
                                            nlayers = 2,
                                            bias=True)

fn = os.path.expanduser('~/Documents/Bilingualism_CVAE/data/tf_outputs/VAE/VAE_weights')
vae.load_weights(fn)

In [4]:
# LOAD CVAE
latent_dim = 16;batch_size = 32;beta = 1;gamma = 100;disentangle = True
cvae, z_encoder, s_encoder, cvae_decoder = get_MRI_CVAE_3D(latent_dim=latent_dim,beta=beta, disentangle=disentangle, gamma=gamma, bias=True, batch_size = batch_size)
loss = list()    

fn = os.path.expanduser('~/Documents/Bilingualism_CVAE/data/tf_outputs/CVAE/CVAE_weights')
cvae.load_weights(fn)

In [5]:
n_samples = 10
fn = os.path.expanduser('~/Documents/Bilingualism_CVAE/data/latent_vecs'+str(n_samples)+'_s.npz')

#just include bilingual participants here
salient_vec_BRAIN = np.array([s_encoder.predict(BRAIN_data[:,:,:,:])[2] for _ in range(n_samples)])
np.savez_compressed(fn, salient_vec_BRAIN=salient_vec_BRAIN)

79/79 [==============================] - 33s 412ms/step


In [6]:
fn = os.path.expanduser('~/Documents/Bilingualism_CVAE/data/latent_vecs'+str(n_samples)+'_z.npz')

background_vec_BRAIN = np.array([z_encoder.predict(BRAIN_data[:,:,:,:])[2] for _ in range(n_samples)])
np.savez_compressed(fn, background_vec_BRAIN=background_vec_BRAIN)
print('done')

79/79 [==============================] - 33s 413ms/step
done


In [7]:
n_samples = 10
fn = os.path.expanduser('~/Documents/Bilingualism_CVAE/data/latent_vecs'+str(n_samples)+'_vae.npz')

vae_vec_BRAIN = np.array([encoder.predict(BRAIN_data[:,:,:,:])[2] for _ in range(n_samples)])
np.savez_compressed(fn, vae_vec_BRAIN=vae_vec_BRAIN)
print('done')

# salient_vec_sfari = np.array([s_encoder.predict(SFARI_data[:,:,:,:])[2] for _ in range(n_samples)])
# background_vec_sfari = np.array([z_encoder.predict(SFARI_data[:,:,:,:])[2] for _ in range(n_samples)])
# vae_vec_sfari = np.array([encoder.predict(SFARI_data[:,:,:,:])[2] for _ in range(n_samples)])


#                     salient_vec_sfari=salient_vec_sfari,
#                     background_vec_sfari=background_vec_sfari,
#                     vae_vec_sfari=vae_vec_sfari)

79/79 [==============================] - 89s 1s/step
done
